# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.7.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Feature Scaling
### Feature Scaling จำเป็น(must) กับ Deep Learning และต้องทำทุกตัวแปรไม่ว่าจะเป็นตัวแปร 1 กับ 0 ก็ตาม

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
print(X_train)

[[-0.99850112  1.71490137 -0.57273139 ... -1.55337352  0.97725852
   0.42739449]
 [ 1.00150113 -0.58312392 -0.57273139 ... -1.55337352 -1.02327069
  -1.02548708]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017  0.97725852
  -0.94479772]
 ...
 [ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017  0.97725852
  -0.14096853]
 [ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017  0.97725852
   0.01781218]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017 -1.02327069
  -1.15822478]]


In [13]:
print(X_test)

[[ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017  0.97725852
  -0.05360571]
 [ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017 -1.02327069
  -0.58392685]
 [ 1.00150113 -0.58312392 -0.57273139 ... -1.55337352  0.97725852
  -0.16685331]
 ...
 [-0.99850112 -0.58312392  1.74601919 ... -1.55337352  0.97725852
   1.0669965 ]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017  0.97725852
   1.13101314]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017  0.97725852
  -0.88790165]]


## Part 2 - Building the ANN

### Initializing the ANN

In [14]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [15]:
# relu = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [17]:
# ถ้า dependent ที่ต้องการหามีแค่ 1 กับ 0 ใช้ units = 1 ได้ แต่ถ้ามีที่ต้องหาคือ A,B,C ก็ทำ OneHotEncoder แล้วใช้ units = 3 ตามจำนวนที่ต้องการ classifier
# ถ้า ใช้ units = 3 ขึ้นไป ใช้ activation เป็น softmax
# ถ้า predict regression ก็ไม่ต้องใส่ activation และ units = 1
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [18]:
#adam = Adaptive Moment Estimation เป็น optimizer ที่เป็นที่นิยมมากที่สุด
#loss สำหรับ classification ที่ predict 2 กลุ่ม(1,0) ใช้ binary_crossentropy แต่ถ้า predict 3กลุ่มขึ้นไป ใช้ categorical_crossentropy
#metrics = ['accuracy'] : focus on accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [19]:
#batch_size : number of predictions compared to the same number of real results (defult = 32)
# Epochs : 1 Epoch หมายความว่า เรา train model โดย feed ข้อมูลเข้า model โดยใช้ข้อมูลใน dataset ที่เรามีอยู่จนครบทั้งหมดทุกตัวเป็นจำนวน 1 รอบ (ใส่เท่าไรก็ได้แต่อย่าน้อยเกินไป)
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5846 - accuracy: 0.7466
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4863 - accuracy: 0.7971
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4560 - accuracy: 0.7976
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4414 - accuracy: 0.8037
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4341 - accuracy: 0.8069
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4284 - accuracy: 0.8098
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4231 - accuracy: 0.8119
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4175 - accuracy: 0.8154
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4107 - accuracy: 0.8174
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4020 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [20]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))

[[0.01634696]]


In [21]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [22]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 0]]


In [23]:
pd.set_option('display.max_row', 80)
data = {'Predict': y_pred.flatten(), 'Actual': y_test}
pd.DataFrame(data)

,Predict,Actual
0,False,0
1,False,0
2,False,0
3,False,0
4,False,0
...,...,...
1995,False,0
1996,True,0
1997,False,0
1998,False,0


### Making the Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1521   64]
 [ 212  203]]


0.862

In [25]:
ann.evaluate(X_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.3303 - accuracy: 0.8620


[0.33030572533607483, 0.8619999885559082]